beep borp

In [ ]:
include("create_jmds.jl")

In [15]:
#https://github.com/FluxML/model-zoo/blob/master/other/iris/iris.jl
using Flux
using Flux: crossentropy, normalise, onecold, onehotbatch
using Statistics: mean

In [10]:
labels = Flux.Data.Iris.labels()
features = Flux.Data.Iris.features()

┌ Info: Downloading iris dataset.
└ @ Flux.Data.Iris C:\Users\chris\.julia\packages\Flux\NpkMm\src\data\iris.jl:24


4×150 Array{Float64,2}:
 5.1  4.9  4.7  4.6  5.0  5.4  4.6  5.0  …  6.8  6.7  6.7  6.3  6.5  6.2  5.9
 3.5  3.0  3.2  3.1  3.6  3.9  3.4  3.4     3.2  3.3  3.0  2.5  3.0  3.4  3.0
 1.4  1.4  1.3  1.5  1.4  1.7  1.4  1.5     5.9  5.7  5.2  5.0  5.2  5.4  5.1
 0.2  0.2  0.2  0.2  0.2  0.4  0.3  0.2     2.3  2.5  2.3  1.9  2.0  2.3  1.8

In [11]:
# Subract mean, divide by std dev for normed mean of 0 and std dev of 1.
normed_features = normalise(features, dims=2)


klasses = sort(unique(labels))
onehot_labels = onehotbatch(labels, klasses)


# Split into training and test sets, 2/3 for training, 1/3 for test.
train_indices = [1:3:150 ; 2:3:150]

X_train = normed_features[:, train_indices]
y_train = onehot_labels[:, train_indices]

X_test = normed_features[:, 3:3:150]
y_test = onehot_labels[:, 3:3:150]

3×50 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1

In [12]:
# Declare model taking 4 features as inputs and outputting 3 probabiltiies, 
# one for each species of iris.
model = Chain(
    Dense(4, 3),
    softmax
)

loss(x, y) = crossentropy(model(x), y)

# Gradient descent optimiser with learning rate 0.5.
optimiser = Descent(0.5)

# Create iterator to train model over 110 epochs.
data_iterator = Iterators.repeated((X_train, y_train), 110)

println("Starting training.")
Flux.train!(loss, params(model), data_iterator, optimiser)

Starting training.


In [13]:
# Evaluate trained model against test set.
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))

accuracy_score = accuracy(X_test, y_test)

println("\nAccuracy: $accuracy_score")

# Sanity check.
@assert accuracy_score > 0.8


Accuracy: 0.94


In [16]:
function confusion_matrix(X, y)
    ŷ = onehotbatch(onecold(model(X)), 1:3)
    y * ŷ'
end
#To avoid confusion, here is the definition of a Confusion Matrix: https://en.wikipedia.org/wiki/Confusion_matrix
println("\nConfusion Matrix:\n")
confusion_matrix(X_test, y_test)


Confusion Matrix:



3×3 Array{Int64,2}:
 16   0   0
  0  16   1
  0   2  15

# Notes
David Banks gives few hour overview of ML
give ML context in notebook

classifying stability of planetary systems?

make ROC curves?

show how optimizer is improving things as it works

show right answer with a linear problem

linear and logistic regression

regulatization terms?

## more specifically...

simple tutorial

show a problem that you dont need optimization to solve, but then solve it with optimization methods

compare iterative methods to analytical

what if we want a regulatization for sparseity/some breaking assumption

everyone will achieve something - have a bonus exploring thing at the end

hand holdy -> advanced (do the same problem with more dimensions to see scaling)

comparing algorithms